In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import nltk
%matplotlib inline

import grequests # You will get errors if grequests is not aboe requests
from bs4 import BeautifulSoup
import lxml
import cchardet
from requests.adapters import HTTPAdapter, Retry
import requests
import re
import pyinputplus as pyip
import requests
import datetime as dt
from datetime import datetime
from tqdm import tqdm
import pickle
import random
import math 
import string

%load_ext autoreload
%autoreload 2

tqdm.pandas()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

In [2]:
# Import other files
%run climbconstants.py
%run unique_route_handling.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\Brayden\Desktop\Giza\.venv\lib\site-packages\grequests.py:22: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util.ssl_ (c:\\Users\\Brayden\\Desktop\\Giza\\.venv\\lib\\site-packages\\urllib3\\util\\ssl_.py)', 'urllib3.util (c:\\Users\\Brayden\\Desktop\\Giza\\.venv\\lib\\site-packages\\urllib3\\util\\__init__.py)']. 
  curious_george.patch_all(thread=False, select=False)
c:\Users\Brayden\Desktop\Giza\.venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


--- IMPORT ---

In [3]:
upload_link = 'https://www.mountainproject.com/user/200180658/brayden-l'
upload_type = 'todo'
df_ulist = download_routelist(upload_type, upload_link)

c:\Users\Brayden\Desktop\Giza\.venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


--- DATA CLEANSE AND STANDARDIZE ---

In [4]:
df_ulist = data_standardize(df_ulist)

c:\Users\Brayden\Desktop\Giza\.venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


--- GRADE HOMOGENIZATION AND ROUTE LENGTH CLEANUP ---

In [5]:
df_ulist = route_length_fixer(df_ulist, 'express')

In [6]:
grade_settings = ['letter', 'even_rand', 'flat', 'even_rand']
df_ulist = grade_homo(df_ulist, *grade_settings)

c:\Users\Brayden\Desktop\Giza\.venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


--- SCRAPE ---

In [7]:
df_ulist = route_scrape_sync(df_ulist)

  0%|          | 0/918 [00:00<?, ?it/s]

: 

: 

--- ANALYZE ---

In [ ]:
df_ulist = extract_tick_details(df_ulist)

100%|██████████| 918/918 [07:09<00:00,  2.14it/s]


In [ ]:
# Let's save the now scraped dataframe to a pickle file
df_ulist.to_pickle('../Data_Archive/df_todo_archive')

In [ ]:
# To load the new pickle file
picklefile = open('../Data_Archive/df_todo_archive', 'rb')
df_ulist = pickle.load(picklefile)

In [ ]:
df_ulist = tick_analysis(df_ulist)

  1%|          | 10/918 [00:00<00:16, 55.45it/s]C:\Users\Brayden\AppData\Local\Temp\ipykernel_15468\3679277498.py:129: RuntimeWarning: invalid value encountered in longlong_scalars
  lead_ratio = tick_counts['Lead']/(tick_counts['Follow'] + tick_counts['TR'] + tick_counts['Lead'])
C:\Users\Brayden\AppData\Local\Temp\ipykernel_15468\3679277498.py:130: RuntimeWarning: invalid value encountered in longlong_scalars
  os_ratio = (tick_counts['Onsight'] + tick_counts['Flash']) / (tick_counts['Onsight'] + tick_counts['Flash'] + tick_counts['Fell/Hung'] + tick_counts['Redpoint'] + tick_counts['Pinkpoint'] + tick_counts['Attempt'] + tick_counts['Send'])
  3%|▎         | 23/918 [00:00<00:25, 35.11it/s]C:\Users\Brayden\AppData\Local\Temp\ipykernel_15468\3679277498.py:130: RuntimeWarning: invalid value encountered in longlong_scalars
  os_ratio = (tick_counts['Onsight'] + tick_counts['Flash']) / (tick_counts['Onsight'] + tick_counts['Flash'] + tick_counts['Fell/Hung'] + tick_counts['Redpoint'] + t

--- SPLIT OUTPUT INTO BOULDER AND ROPED SUBFRAME ---

In [ ]:
# Select Boulder or Route
df_ulist_r = df_ulist[df_ulist['Route Type'] != 'Boulder']
df_ulist_b = df_ulist[df_ulist['Route Type'] == 'Boulder']

In [ ]:
# Filter

In [ ]:
min_ticks = 30

# Rarely led
df_low_lead = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['Lead Ratio'] < 0.4) & (df_ulist_r['Pitches'] == 1)].sort_values(by='Lead Ratio')

# Rarely toproped
df_high_lead = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['Lead Ratio'] > 0.9) & (df_ulist_r['Pitches'] == 1)].sort_values(by='Lead Ratio', ascending=False)

# Hard to OS
df_low_OS_r = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['OS Ratio'] < 0.35)].sort_values(by='OS Ratio')
df_low_OS_b = df_ulist_b[(df_ulist_b['Num Ticks'] >= min_ticks) & (df_ulist_b['OS Ratio'] < 0.35)].sort_values(by='OS Ratio')

# High OS
df_high_OS_r = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['OS Ratio'] > 0.8)].sort_values(by='OS Ratio', ascending=False)
df_high_OS_b = df_ulist_b[(df_ulist_b['Num Ticks'] >= min_ticks) & (df_ulist_b['OS Ratio'] > 0.8)].sort_values(by='OS Ratio', ascending=False)

# Find route on list that is hardest and easiest to OS given a grade
OS_cutoff_num = 3


In [ ]:
grouplist = df_ulist['Rating'].unique()
outlist = []
for group in grouplist:
    outlist.extend(list(df_ulist[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist['Rating'] == group)].nsmallest(10, 'OS Ratio').index))
df_ulist.loc[outlist].sort_values('OS Ratio')

,Route,Original Rating,Rating,URL,Pitches,Location,Avg Stars,Your Stars,Route Type,Your Rating,Length,Rating Code,Route ID,Risk,Re Mainpage,Re Statpage,Route Ticks,Num Ticks,Num Tickers,Lead Ratio,OS Ratio,Repeat Sender Ratio,Mean Attempts To RP,Tick Counts
736,Galen's Crack,5.10c,5.10c,https://www.mountainproject.com/route/10767604...,1,California > Yosemite National Park > Tuolumne...,3.1,-1,Trad,NaN,45.0,3200,107676048,NaN,<Response [200]>,<Response [200]>,Username ...,127.0,102.0,0.091837,0.000000,1.000000,1.000000,TR 89 Lead 9 Fell/Hung ...
902,Desert Gold,5.13a,5.13a,https://www.mountainproject.com/route/10593491...,2,Nevada > Southern Nevada > Red Rocks > 16-Blac...,4.0,-1,Trad,NaN,150.0,8600,105934919,PG13,<Response [200]>,<Response [200]>,Username ...,51.0,43.0,0.900000,0.000000,1.000000,1.444444,Lead 36 Fell/Hung 24 Redpoint ...
877,Make or Break Flake,5.10b,5.10b,https://www.mountainproject.com/route/10572552...,1,California > Joshua Tree National Park > Quail...,2.3,-1,Trad,NaN,60.0,2900,105725521,NaN,<Response [200]>,<Response [200]>,Username ...,39.0,35.0,0.466667,0.000000,1.500000,1.000000,Lead 7 TR 4 Follow 4 F...
333,Crash and Burn,5.12d,5.12d,https://www.mountainproject.com/route/10662024...,1,California > Los Angeles Basin > Santa Monica ...,3.7,-1,Sport,NaN,50.0,7500,106620240,NaN,<Response [200]>,<Response [200]>,Username ...,96.0,63.0,0.977778,0.016129,1.000000,2.764706,Lead 88 Fell/Hung 44 Redpoint 1...
649,Brother From Another Planet,5.12-,5.12a,https://www.mountainproject.com/route/10605839...,1,Utah > Southeast Utah > Indian Creek > Fin Wall,3.8,-1,Trad,NaN,90.0,6700,106058396,NaN,<Response [200]>,<Response [200]>,Username ...,53.0,44.0,0.586957,0.038462,1.142857,1.000000,Lead 27 TR 19 Fell/Hung 1...
203,Pow! Right in the Kisser,5.12a,5.12a,https://www.mountainproject.com/route/10702785...,1,California > Los Angeles Basin > Santa Monica ...,3.0,-1,Sport,NaN,40.0,6600,107027859,NaN,<Response [200]>,<Response [200]>,Username ...,333.0,201.0,0.874172,0.053719,1.086022,1.537500,Lead 264 Fell/Hung 141 Redpoint ...
893,Separate Reality,5.12a,5.12a,https://www.mountainproject.com/route/10587459...,1,California > Yosemite National Park > Yosemite...,4.0,-1,Trad,NaN,50.0,6600,105874590,NaN,<Response [200]>,<Response [200]>,Username ...,201.0,163.0,0.877301,0.053846,1.023256,1.472222,Lead 143 Fell/Hung 86 Redpoint ...
177,Ground Zero,5.12a,5.12a,https://www.mountainproject.com/route/10607525...,1,California > Los Angeles Basin > Santa Monica ...,3.0,-1,Sport,NaN,35.0,6600,106075250,NaN,<Response [200]>,<Response [200]>,Username ...,226.0,147.0,0.823529,0.060976,1.064935,1.671642,Lead 168 Fell/Hung 82 Redpoint ...
591,Way Rambo,5.12-,5.12a,https://www.mountainproject.com/route/10571864...,1,Utah > Southeast Utah > Indian Creek > Way Rambo,3.8,-1,Trad,NaN,100.0,6700,105718642,NaN,<Response [200]>,<Response [200]>,Username ...,858.0,649.0,0.526144,0.061644,1.026667,1.465517,Lead 322 TR 261 Fell/Hung ...
33,Dawn of an Error,5.11+,5.11c,https://www.mountainproject.com/route/10661202...,1,Utah > Southeast Utah > Potash Road > Long Can...,3.7,-1,Trad,NaN,115.0,5300,106612025,NaN,<Response [200]>,<Response [200]>,Username ...,36.0,28.0,0.681818,0.066667,1.000000,1.000000,Lead 15 Fell/Hung 13 TR ...
